In [0]:
from pyspark.sql import SparkSession
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
spark = SparkSession.builder.appName("ReadCSV1").getOrCreate()
df = spark.read.csv("/FileStore/tables/Question_Answer-1.csv", header=True, inferSchema=True)
df=df.select("Paragraph").distinct().rdd.map(lambda x : x[0]).collect()
#print(df)
stop_words = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'aren\'t', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'can\'t', 'cannot', 'could', 'couldn\'t', 'did', 'didn\'t', 'do', 'does', 'doesn\'t', 'doing', 'don\'t', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn\'t', 'has', 'hasn\'t', 'have', 'haven\'t', 'having', 'he', 'he\'d', 'he\'ll', 'he\'s', 'her', 'here', 'here\'s', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'how\'s', 'i', 'i\'d', 'i\'ll', 'i\'m', 'i\'ve', 'if', 'in', 'into', 'is', 'isn\'t', 'it', 'it\'s', 'its', 'itself', 'let\'s', 'me', 'more', 'most', 'mustn\'t', 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', 'shan\'t', 'she', 'she\'d', 'she\'ll', 'she\'s', 'should', 'shouldn\'t', 'so', 'some', 'such', 'than', 'that', 'that\'s', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'there\'s', 'these', 'they', 'they\'d', 'they\'ll', 'they\'re', 'they\'ve', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'wasn\'t', 'we', 'we\'d', 'we\'ll', 'we\'re', 'we\'ve', 'were', 'weren\'t', 'what', 'what\'s', 'when', 'when\'s', 'where', 'where\'s', 'which', 'while', 'who', 'who\'s', 'whom', 'why', 'why\'s', 'with', 'won\'t', 'would', 'wouldn\'t', 'you', 'you\'d', 'you\'ll', 'you\'re', 'you\'ve', 'your', 'yours', 'yourself', 'yourselves']
vectorizer1 = TfidfVectorizer(ngram_range=(1,2))
tfidf1 = vectorizer1.fit_transform(df)
def recommendparagraphs1(query, k):
    query_tfidf1 = vectorizer1.transform([query])
    scores = cosine_similarity(query_tfidf1, tfidf1).flatten()
    sort = np.argsort(scores)[::-1]
    return [df[i] for i in sort[:k]]
def pararecommendation(query, k):
    recom = recommendparagraphs1(query, k)
    print("Recommendations:")
    for i, rec in enumerate(recom):
        print(f"{i+1}. {rec}")


df1 = spark.read.csv("/FileStore/tables/QA1.csv", header=True, inferSchema=True)

p=df1.select("Combined").rdd.map(lambda x : x[0]).collect()
#print(p)
my_list = list(filter(lambda x: x is not None, p))
new_list = [string.replace('[','').replace(']','') for string in my_list]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# random paragraphs
paragraphs = new_list

# create the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# fit the vectorizer on the paragraphs
tfidf = vectorizer.fit_transform(paragraphs)

# function to recommend paragraphs based on a query message
def recommend_paragraphs(query, k=1):
    # transform the query using the TF-IDF vectorizer
    query_tfidf = vectorizer.transform([query])
    
    # calculate the cosine similarity between the query and the paragraphs
    similarity_scores = cosine_similarity(query_tfidf, tfidf).flatten()
    
    # sort the paragraphs by their similarity score to the query and return the top-k
    sorted_indexes = np.argsort(similarity_scores)[::-1]
    return [paragraphs[i] for i in sorted_indexes[:k]]

query = "What does do previous games do to defeated companions?"

recommendations = recommend_paragraphs(query, k=1)
print(f"Query: {query}")
for i, rec in enumerate(recommendations):
    if("" == (rec.split('?')[1].strip())):
        print("Searching for Paragraph")
        words = query.split()
        filtered_sentence = []
        for word in words:
            if word not in stop_words:
                filtered_sentence.append(word)
        filtered_sentence = ' '.join(filtered_sentence)
        print(filtered_sentence)
        pararecommendation(filtered_sentence, 4)
    else:
        print(rec.split('?')[1].strip())

Query: What does do previous games do to defeated companions?
Searching for Paragraph
What previous games defeated companions?
Recommendations:
1. The artificial intelligence (AI) of enemies in Twilight Princess is more advanced than that of enemies in The Wind Waker. Enemies react to defeated companions and to arrows or slingshot pellets that pass by, and can detect Link from a greater distance than was possible in previous games.
2. "Melbourne is notable as the host city for the 1956 Summer Olympic Games (the first Olympic Games held in the southern hemisphere and Oceania, with all previous games held in Europe and the United States), along with the 2006 Commonwealth Games. Melbourne is so far the southernmost city to host the games. The city is home to three major annual international sporting events: the Australian Open (one of the four Grand Slam tennis tournaments); the Melbourne Cup (horse racing); and the Australian Grand Prix (Formula One). Also, the Australian Masters golf to